# Freespace Garment Simulation

Notebook for simulating garments moving in freespace. This will ultimately be used to generate training data for the tracking model we're putting together.

**Note:** We probably want to have two modes:
- Simple dynamics where the cloth behaves like a quasi-rigid object.
  - This should be easy-peasy for the network to track since it's essentially a rigid body dynamics problem if the gripper is moving slowly enough.
- More complicated dynamics where the cloth deforms due to the "gripper" moving faster than the cloth can reach a quasi-static state.
  - This will be harder for the network to predict but it should still be possible.

Ultimately we probably won't want hard distinctions between the two modes of operation but splitting them like this will help me to develop the functions/classes necessary to write the data generation pipeline.

## TODO

- ~~Set animation interpolation scheme to 'linear' to simplify things here.~~
  - ~~Seems like you can do `keyframe_variable.interpolation = "LINEAR"`~~
  - Actually, smooth motion is probably preferable. Linear is too sharp.
    - Can maybe edit F-Curve properties to make it sharper if necessary?

In [1]:
import os
import sys
from pathlib import Path

import bpy
import mathutils
import numpy as np

# Imports from this repository
sys.path.append("../../")
sys.path.append("../")
from simulation.cloth_3d_util.accessor import Cloth3DCanonicalAccessor
from simulation.cloth_3d_util.util import loadInfo
from simulation.pipeline.simulate_garment_hanging_rest_state import \
    simulate_garment_hanging_rest_state
from simulation.blender_util_dylan.physics import set_sim_output_as_default_mesh_shape
from simulation.blender_util_dylan.checkpointer import BlendFileCheckpointer
from simulation.blender_util_dylan.gripper_animation import GripperAnimation

%load_ext autoreload
%autoreload 2

WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.


/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  retur

In [2]:
FILE_ROOT = Path(os.getcwd())
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))
OUTPUT_ROOT = FILE_ROOT / ".." / "script_output" / "sim_pipeline_driver_test"

# Make the output directory if it doesn't exist.
OUTPUT_ROOT.mkdir(exist_ok=True)


PLANE_OFFSET = 0.025  # [m]

In [3]:
sample_configuration = {
    "sample_id": "00380",
    "garment_name": "Tshirt",
    "grip_vertex_idx": 0
}

smpl_simulation_duration_pair = (0, 120)

sample_configs = [sample_configuration]

## Start With Simple Freespace Dynamics

This is where the garment behaves much like a rigid body during motion, meaning the gripper will have to move fairly slowly.

I think this should be packaged up to where we have a routine for simulating one freespace motion in one 3D vector direction. Doing anything more complicated would introduce the possibility of complicated dynamics.

I just baked a garment simulation in Blender and moving in direction $[0, 0, -1]^T$ over 100 frames with a velocities:
- $0.0016 / \text{frame}$ was fine.
- $0.0032 / \text{frame}$ was fine.
- $0.0064 / \text{frame}$ looked okay. I suspect that if it was in the $X$ or $Y$ direction it would be less okay.

Moving in $[1, 0, 0]^T$ direction.
- $0.0064 / \text{frame}$ looked fine even with linear interpolation.

In [4]:
# Example of what the freespace dynamics arguments could look like.
# grip_lowering_args = {
#     "initial_sim_end_frame": smpl_simulation_duration_pair[1],
#     "start_frame": 1,
#     "end_frame": 100,
#     "fraction_lowered": 0.25, # Fraction of the cloth that will be lowered onto the table.
# }

# class FreespaceSimpleSimArgs:
#     def __init__(self, direction_vec: np.ndarray, velocity: float, )

# freespace_sim_args = {
#     "direction": np.array((1.0, 0.0, 0.0))
# }



### See if Cheng's Gripper Animation Utilities Still Work With New Blender Versions

In [5]:
## Setup the simulation.
accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)
config = sample_configs[0]
sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}_simple_dynamics"
sample_dir = OUTPUT_ROOT / sample_key
sample_dir.mkdir(exist_ok=True)

result_file = "simulation_result.pk"
result_path = sample_dir / result_file

# Get a dictionary containing the data for this sample garment.
sample_data = accessor.get_sample_data(**config)

garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
garment_info = loadInfo(garment_info_mat_filename)

# if result_path.exists():
#     print(f"WARNING! Result file, '{result_path}', already exists! Skipping this run. Reading result instead")
#     result_data = pickle.load(result_path.open('rb'))

checkpointer = BlendFileCheckpointer(sample_dir, save_new_checkpoints=True)

In [22]:
# Confirmed to work as expected!
if checkpointer.does_rest_state_checkpoint_exist():
    print("Hanging state checkpoint exists. Loading checkpoint from file.")
    # Load the checkpoint instead of simulating the resting state again.
    checkpointer.load_hanging_rest_state()
else:
    print("Hanging state checkpoint does not exist. Simulating and saving checkpoint.")
    # Run the SMPL simulation to get the garment in a stationary hanging configuration
    frames_to_resting_state = 120
    result_data_smpl = simulate_garment_hanging_rest_state(config, sample_data, frames_to_resting_state)

    # Then set the default mesh shape to this shape
    cloth_obj = bpy.data.objects["cloth"]
    set_sim_output_as_default_mesh_shape(cloth_obj, frames_to_resting_state)

    # Now create a checkpoint for this so we don't have to resimulate every time we run this
    # notebook.
    checkpointer.save_hanging_rest_state()

Hanging state checkpoint exists. Loading checkpoint from file.
Read blend: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/runners/../script_output/sim_pipeline_driver_test/00380_Tshirt_0_simple_dynamics/hanging_rest_state.blend
Successfully reloaded hanging rest state checkpoint.


## Planning Dynamics Routines

Now that we've simulated the garment hanging rest state and set that as the default mesh shape, we can start doing dynamics runs.

We will control the motion of the "gripper" (empty object that the cloth vertices are pinned to) by controlling the keyframes - the trajectory points - and the F-curves - the interpolation method between keyframes.

For Blender, the following is required for the definitions:
- Each keypoint needs to be defined by:
    - Object origin location
    - Frame with which to insert the keyframe
- F-curves - default is Bezier with pretty reasonable parametrs
  - For linear control, no parameters are needed.
  - For other interpolation methods I'm unsure.

**TODO:** Decide on Fcurve interpolation scheme (linear or not?). Non-linear seems preferable?
- Let's just use the default F-Curves and get as far as we can with this setting.
**TODO:** Decide on programmatic representation. Velocity control with a unit direction vector seems reasonable.

For keyframe insertion, can potentially follow this pattern:
```python
obj = bpy.context.active_object  # Or however else we get the object pointer
for keyframe_frame, obj_location in (keyframes):
    obj.location = obj_location  # I believe this can be a flat numpy array
    obj.keyframe_insert(data_path="location",
                        index=-1,  # indicates to place the keyframe last
                        group=group,  # group name F-curve added to if it doesn't exist
                        frame=keyframe_frame)
```

In [23]:
# Manually add an initial keyframe to test out the GripperAnimation class
cloth_obj = bpy.data.objects["cloth"]
# cloth_obj.animation_data_clear()
# cloth_obj.keyframe_insert(data_path="location", frame=0, group="gripper_action_1")

In [24]:
# print(cloth_obj.animation_data.action.fcurves)
# [print(a) for a in cloth_obj.animation_data.action.fcurves]
gripper_animation = GripperAnimation(cloth_obj)

# for fcurve in cloth_obj.animation_data.action.fcurves:
#     print(type(fcurve))
#     print("fcurve array_index:", fcurve.array_index)
#     print(fcurve.data_path)
#     print(fcurve.group.name)
#     # print(fcurve.keyframe_points[0].co)  # Same thing as what's shown below
#     print(len(fcurve.keyframe_points))
#     for keyframe in fcurve.keyframe_points:
#         print('\t', keyframe.co)

Starting gripper keyframe frame: 0


In [25]:
print(cloth_obj.location)
print(bpy.context.scene.frame_current)
# bpy.context.scene.frame_set(0)
# print(cloth_obj.location)
# print(bpy.context.scene.frame_current)

<Vector (0.0473, 0.0639, -0.1397)>
0


In [26]:
direction_vec = np.array((1.0, 0.0, 0.0))
# velocity = 0.0064  # in old units of m / frame
velocity_mps = 0.15  # [meters / second]
gripper_animation.add_movement(direction_vec, velocity_mps, end_frame=50)

direction_vec = np.array((0.0, -1.0, 0.0))
gripper_animation.add_movement(direction_vec, velocity_mps, end_frame=100)

In [27]:
def print_object_keyframe_locations(obj: bpy.types.Object):
    loc_idx_map = {0: 'x', 1: 'y', 2: 'z'}
    locs = {'x': [], 'y': [], 'z': []}
    keyframe_frames = {'x': [], 'y': [], 'z': []}

    for fcurve in obj.animation_data.action.fcurves:
        axis_key = loc_idx_map[fcurve.array_index]
        coordinate_locations = locs[axis_key]
        coordinate_frames = keyframe_frames[axis_key]

        for pt in fcurve.keyframe_points:
            # TODO: The pt.co is two values instead of one? Maybe the first value is the frame.
            # The second value is definitely the cartesian coordinate.
            coordinate_frames.append(pt.co[0])
            coordinate_locations.append(pt.co[1])
            

    print("Locations:", locs)

In [28]:
print_object_keyframe_locations(cloth_obj)

Locations: {'x': [0.04728107154369354, 0.35978108644485474, 0.35978108644485474], 'y': [0.06388848274946213, 0.06388848274946213, -0.24861150979995728], 'z': [-0.1396816223859787, -0.1396816223859787, -0.1396816223859787]}


In [29]:
bpy.ops.wm.save_as_mainfile(filepath=(sample_dir / "gripper_animation_check.blend").as_posix())

Info: Saved "gripper_animation_check.blend"


{'FINISHED'}

## Complicated Dynamics

I **may** want to animate the "gripper" on a Bezier curve to make introducing non-rigid behavior into the cloth easier. 

Another (simpler) option is to just move the gripper faster in a single direction.
**Note:** I imagine the *impulse* is what's important here. Quickly changing the velocity of the gripper.
- Due to this, I might have to change the Fcurve for the "gripper" motion to linear so that it quickly jerks the cloth instead of the typical smooth interpolation scheme.